This file is for creating the dataset for testing

In [1]:
# Importing libraries
import os
import pickle
import pandas as pd
import numpy as np
from skimage.transform import resize
from skimage.io import imread

In [2]:
# Importing the Database (local machine)
import os

dataDir = os.path.join('..', 'dataset', 'ExtractedSet', 'dataset')

# Verify the paths exist
print(f"Data directory path: {os.path.abspath(dataDir)}")
print(f"Data directory exists: {os.path.isdir(dataDir)}")

Data directory path: /Users/yukyunglee/Documents/GitHub/cs306_project2/dataset/ExtractedSet/dataset
Data directory exists: False


In [3]:
#Imports
import os

#The save the directory of where all the pictures are
picDirectory = "../dataset/ExtractedSet/"

#Folder name of the dataset
fileName = "dataset"

#Compile the data path
picPath = os.path.join(picDirectory, fileName)

#Rename the files within the data directory
categories = ["green", "sheep", "slow", "stop", "yellow", "free"]
for classes in categories:
    print(f"renaming... category : {classes}")
    path = os.path.join(dataDir, classes)

    #Check if path exists
    if not os.path.isdir(path):
        print(f"Directory not found: {path}")
        continue

    #Set up the files in the folder
    files = os.listdir(path)

    #Loop through the image folder and renaming each file
    for i, filename in enumerate(files, start = 1):
        newFileName = f"{classes}{i}.jpg"
        newPath = os.path.join(path, newFileName)
        
        # Skip file renaming if file already exists
        if os.path.exists(newPath):
            print(f"{filename} already exists")
            continue
        oldPath = os.path.join(path, filename)
        os.rename(oldPath, newPath)
    
    print(f"renaming {classes} finished")



renaming... category : green
Directory not found: ../dataset/ExtractedSet/dataset/green
renaming... category : sheep
Directory not found: ../dataset/ExtractedSet/dataset/sheep
renaming... category : slow
Directory not found: ../dataset/ExtractedSet/dataset/slow
renaming... category : stop
Directory not found: ../dataset/ExtractedSet/dataset/stop
renaming... category : yellow
Directory not found: ../dataset/ExtractedSet/dataset/yellow
renaming... category : free
Directory not found: ../dataset/ExtractedSet/dataset/free


In [4]:
flat_data_arr = []
target_arr = []
categories = ["green", "sheep", "slow", "stop", "yellow", "free"]

# Flattening the images into an array
for classes in categories:
  print(f'loading... category : {classes}')
  path = os.path.join(picDirectory,"dataset" ,classes)
  
  # Check if path exists
  if not os.path.isdir(path):
    print(f"Directory not found: {path}")
    continue
  
  counter = 0
  for img in os.listdir(path):
    img_path = os.path.join(path, img)
    
    # Skip folders - only process image files
    if not os.path.isfile(img_path):
      continue
    
    # Skip non-image files
    if not img.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
      continue
    
    img_array = imread(img_path)
    img_array = np.squeeze(img_array)
    img_resized = resize(img_array, (224, 224, 3))
    
    # Append flattened image to list
    flat_data_arr.append(img_resized.flatten())
    
    # Append class label to list
    target_arr.append(classes)
    
    counter += 1
  
  print(f'loaded category {classes} : {counter} images successfully')

# Convert lists to NumPy arrays
flat_data = np.array(flat_data_arr)
target = np.array(target_arr)

print(f"flat_data shape: {flat_data.shape}")
print(f"target shape: {target.shape}")

# Compiling the data frame for the training
df = pd.DataFrame(flat_data)
df['Target'] = target
print("Data Frame created")
print(f"DataFrame shape: {df.shape}")

loading... category : green
Directory not found: ../dataset/ExtractedSet/dataset/green
loading... category : sheep
Directory not found: ../dataset/ExtractedSet/dataset/sheep
loading... category : slow
Directory not found: ../dataset/ExtractedSet/dataset/slow
loading... category : stop
Directory not found: ../dataset/ExtractedSet/dataset/stop
loading... category : yellow
Directory not found: ../dataset/ExtractedSet/dataset/yellow
loading... category : free
Directory not found: ../dataset/ExtractedSet/dataset/free
flat_data shape: (0,)
target shape: (0,)
Data Frame created
DataFrame shape: (0, 2)


In [7]:
# Save DataFrame to a local file
import pandas as pd
import os

# Create the directory if it doesn't exist

# Specify the local path where you want to save the file
# Make sure you have write permissions to this directory
FramePath = "./DataFrame/testdataframe.pkl"
os.makedirs(os.path.dirname(FramePath), exist_ok=True)


try:
    df.to_pickle(FramePath)
    print(f"DataFrame successfully saved to {FramePath}")
except PermissionError:
    print(f"Permission denied: Unable to save the file to {FramePath}. Please check the directory path and your file permissions.")
except Exception as e:
    print(f"An error occurred: {e}")

DataFrame successfully saved to ./DataFrame/testdataframe.pkl
